In [1]:
import PublicDataReader as pdr
from config import OpenAPI
import pandas as pd

# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

print(pdr.__all__)
print(pdr.__version__)
# print(pdr.__info__)

['__version__', '__info__', 'Transaction', 'Building', 'code_list']
2021.11.


In [2]:
# 법정동코드 목록
code = pdr.code_list()
code.head()

,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,NaN,1100000000,NaN,NaN,19880423,NaN
1,11,서울특별시,11110,종로구,1111000000,NaN,NaN,19880423,NaN
2,11,서울특별시,11110,종로구,1111010100,청운동,NaN,19880423,NaN
3,11,서울특별시,11110,종로구,1111010200,신교동,NaN,19880423,NaN
4,11,서울특별시,11110,종로구,1111010300,궁정동,NaN,19880423,NaN


In [3]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Transaction(serviceKey, debug=True)

[INFO] 아파트 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 아파트 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 오피스텔 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 오피스텔 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 단독다가구 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 단독다가구 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 연립다세대 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 연립다세대 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 상업업무용 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 토지 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 분양입주권 매매 조회 서비스 정상 - (00) NORMAL SERVICE.


In [ ]:
prod="아파트"
trans="매매"
sigunguCode="41135"
yearMonth="202101"

In [ ]:
df = self.read_data(prod, trans, sigunguCode, yearMonth)
df

In [ ]:
df = self.collect_data(prod, trans, sigunguCode, "202101", "202103")
df

## Building Class

In [4]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Building(serviceKey, debug=True)

[INFO] 기본개요 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 총괄표제부 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 표제부 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 층별개요 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 부속지번 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 전유공용면적 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 오수정화시설 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 주택가격 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 전유부 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 지역지구구역 조회 서비스 정상 - (00) NORMAL SERVICE.


In [7]:
self.metaDict['전유공용면적']['parameters']

['sigunguCd',
 'bjdongCd',
 'platGbCd',
 'bun',
 'ji',
 'startDate',
 'endDate',
 'dongNm',
 'hoNm']

In [5]:
category = "기본개요"

sigunguCd = "41135"
bjdongCd = "11000"
bun = "0560"
ji = "0003"

df = self.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
df

,법정동코드,건물명,블록,번,외필지수,생성일자,구역코드,구역코드명,지,지구코드,지구코드명,지역코드,지역코드명,로트,관리건축물대장PK,관리상위건축물대장PK,새주소법정동코드,새주소본번,새주소도로코드,새주소부번,새주소지상지하코드,도로명대지위치,대지구분코드,대지위치,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,순번,시군구코드,특수지명
0,11000,,,0560,0,20190530,301,제1종지구단위계획구역,0003,,,,,,41135-100202741,,11001,34.0,411354340524,0.0,0,경기도 성남시 분당구 판교역로2번길 34,0,경기도 성남시 분당구 백현동 560-3번지,1,일반,2,일반건축물,1,41135,


## Transaction Class

In [ ]:
# 11종 데이터 조회 테스트
sigunguCode = "11680"
yearMonth = "202110"

df_list = []
for prod in self.metaDict.keys():
    for trans in self.metaDict[prod].keys():
        print()
        print("▶", prod, trans)
        print()
        df= self.read_data(prod, trans, sigunguCode, yearMonth)
        df_list.append(df)
        print(df.head(1))

In [ ]:
prod = "아파트"
trans = "매매"
sigunguCode = "11680"
yearMonth = "202110"
startYearMonth = "202101"
endYearMonth = "202103"

In [ ]:
df = self.read_data(prod, trans, sigunguCode, yearMonth)
df

In [ ]:
df_sum = self.collect_data(prod, trans, sigunguCode, startYearMonth, endYearMonth)
df_sum

## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

### Features

- 공통

    - 로깅 기능
        - 세션 초기화 시 `debug` 인자 추가
            - False(Default): 오류 레벨만 출력
            - True: 모든 레벨 출력
        - OPEN API 에러 코드 출력

    - Air Korea Open API 모듈 제거
    
    - 법정동코드 테이블 조회 함수 추가

- 부동산 실거래가

    - 데이터 조회 메서드 일원화
        - molit.read_data('아파트', '매매', '11680', '202111')
            - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가
    - 데이터 수집 메서드 신규 메서드 추가
        - molit.collect_data('아파트', '매매', '11680', '202101', '202103')

- 건축물대장
    
    - 데이터 조회 메서드 일원화
        - molit.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)